# Get all peaks BED

In [1]:
# Get BEDs prepared for input

dir_beds=/data/hodges_lab/Tim/biomodal/data/tcseq/

## Dynamic: 

cd ${dir_beds}

# Combine
# Dynamic only
cat peak_subset_cluster_1.bed \
peak_subset_cluster_2.bed \
peak_subset_cluster_3.bed \
peak_subset_cluster_4.bed \
peak_subset_cluster_5.bed \
peak_subset_cluster_6.bed \
peak_subset_cluster_7.bed \
peak_subset_static.bed | \
bedtools sort -i - | bedtools merge -i - > peaks_all.txt

# Libraries

In [1]:
#  /data/hodges_lab/Tim/.conda/envs/jupyter_modality/lib/python3.11/site-packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyranges as pr
import seaborn as sns
import statsmodels.api as sm
import xgboost as xgb
import xarray as xr
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
import pysam
import os
os.environ["DS_BUCKET_CACHE"]="/data/hodges_lab/Tim/biomodal/tmp/"
import math
import matplotlib as mpl


import modality
from modality.contig_dataset import ContigDataset
from modality.annotation import get_tss_region

import warnings
warnings.filterwarnings(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

biomodal_palette = ["#003B49", "#9CDBD9", "#F87C56", "#C0DF16", "#05868E"]

print("Loaded.")

2024-06-27 11:00:51 | INFO | [/data/hodges_lab/Tim/.conda/envs/jupyter_modality/lib/python3.11/site-packages/modality/read_locally.py:20] Setting cache dir as /data/hodges_lab/Tim/biomodal/tmp/. To modify alter the DS_BUCKET_CACHE environment variable
Loaded.


# Load 6-base duet evoC data

In [2]:
## Load 6-base duet evoC data
sixbase_path = "/data/hodges_lab/Tim/biomodal/data/Hodges_2024_02_CG.zarrz"
ds = ContigDataset.from_zarrz(sixbase_path)

# Load region file

In [3]:
dir_regions="/data/hodges_lab/Tim/biomodal/data/tcseq/"

## Load

In [4]:
# set groups to iterate through 
# note: the files were set up to have a common name, except for groupnames to iterate
sets=['all']

In [5]:
# create a dictionary to keep track of input BED files
beds={}

# load in BED files to the dictionary
for i in sets:
    beds[i]=pr.readers.read_bed(dir_regions + "peaks_"+str(i)+".txt")

In [6]:
beds

{'all': +---------------------+-----------+-----------+
 | Chromosome          | Start     | End       |
 | (category)          | (int64)   | (int64)   |
 |---------------------+-----------+-----------|
 | chr1                | 267930    | 268098    |
 | chr1                | 629030    | 634961    |
 | chr1                | 778344    | 779234    |
 | chr1                | 827231    | 827715    |
 | ...                 | ...       | ...       |
 | chrUn_KI270747v1    | 3652      | 3819      |
 | chrUn_KI270752v1    | 20582     | 20719     |
 | chrUn_KI270754v1    | 36001     | 36168     |
 | chrX_KI270880v1_alt | 5139      | 5600      |
 +---------------------+-----------+-----------+
 Unstranded PyRanges object has 101,213 rows and 3 columns from 98 chromosomes.
 For printing, the PyRanges was sorted on Chromosome.}

# Center bed files

## Function def

In [7]:
## center_bed_coord
## Purpose: Take in a pyranges bed file and centers the start and end coordinates
##          to represent one base pair
## Takes: PyRanges object
## Returns: PyRanges object
## E.g.: chr1  100  200  -->  chr1  150 151
def center_bed_coord(input_bed, offset):
    # make copy of input PyRanges as df
    input_bed_df=input_bed.df
    # find start
    input_bed_df['Start']=np.floor((input_bed.df['Start']+input_bed.df['End']) / 2).astype(int)-offset
    # find end
    input_bed_df['End'] = (np.floor((input_bed.df['Start']+input_bed.df['End']) / 2)).astype(int)+offset
    # convert to PyRanges object
    input_bed_pr=pr.PyRanges(input_bed_df)
    # return PyRanges object
    return(input_bed_pr)

## center

In [8]:
# create empty dictionary to keep track of centered BEDs
beds250={}
offset_val=250

# center
for i in sets:
    beds250[i]=center_bed_coord(beds[i], offset_val)

# Map ATAC_6base data

## Function def

In [9]:
## Written by Biomodal
def get_atac_6base_over_tss(time, ds, tss_pr):

    # select sample for the timepoint
    if time == 0:
        sample = "10274-AJ-0289"
        atac_time = "0hr"
    elif time == 4:
        sample = "10274-AJ-0290"
        atac_time = "108hr"
    elif time == 8:
        sample = "10274-AJ-0291"
        atac_time = "12day"

    print("Getting 6-base data for sample: " + sample)
    ds = ds.select_samples([sample])
    
    ds = ds.assign_fractions(
        numerators=["num_modc", "num_mc", "num_hmc"],
        denominator="num_total_c", 
        min_coverage=10
    )

    # load ATAC_seq data
    atac_path = "/data/hodges_lab/Tim/biomodal/data/bedgraphs/NPCdiffATACme" + atac_time + "_merge_filtered.bedGraph"
    print("Loading ATAC-seq data from: " + atac_path)
    atac = pd.read_csv(atac_path, sep = "\t",
                       names=['Chromosome', 'Start', 'End', 'Accessibility'])
    
    contigs = ["chr" + str(x) for x in list(range(1,23)) + ["X", "Y"]]
    atac = atac[atac.Chromosome.isin(contigs)]
    atac.Chromosome = pd.Categorical(atac.Chromosome)
    atac.Start = atac.Start.astype(int)
    atac.End = atac.End.astype(int)
    atac.Accessibility = atac.Accessibility.astype(float)
    atac['End'] = atac['End'] - 1
    # make as pyranges
    atac_pr = pr.PyRanges(atac)

    print("Overlapping ATAC-seq data with TSS regions.")
    # find atac-regions that overlap tss
    accessib = atac_pr.intersect(tss_pr)
    # assign id to the overlapping regions
    # if less than 1 bp apart, assign the same id
    r = accessib.cluster(slack=1)
    # add length of regions
    r.Length = r.lengths()
    # group by clusters
    g = r.df.groupby('Cluster')

    # start and end coordinates for each cluster is now a TSS
    # for each region, get the mean accessibility, weighted by the length of the region
    result = pr.from_dict(
            {
                "Chromosome": g.Chromosome.first(),
                "Start": g.Start.min(),
                "End": g.End.max(),
                "accessibility": g.apply(collapse, 'Accessibility'),
            }
        )
    
    # join with tss data
    joined_list = result.join(tss_pr)
    atac_over_tss = joined_list
    atac_over_tss = atac_over_tss.drop_duplicate_positions().sort().copy()
    
    # update Range ID
    atac_over_tss.Ranges_ID = list(range(len(atac_over_tss)))
    
    # summarise 6-base data for all TSS regions
    print("Summarising 6-base data over TSS regions.")
    sixbase_over_tss = []
    
    for mod in ["mc", "hmc", "modc"]:
        print(mod)
    
        for offset_val in (-1000, -500, 0, 500, 1000):
            
            qq = atac_over_tss.apply(offset, offset=offset_val).unstrand()
                    
            y = ds.reduce_byranges(
                ranges=qq[["Chromosome", "Start", "End", "Ranges_ID"]],
                var=[f'frac_{mod}', f'num_{mod}'],
                min_count=1,
                # min_count is the minimum number of data entries required in each interval of range to calculate mean
                # we could adjust this to 3 or so
            )
    
            # also add num_contexts and range_length as data_vars
            y = y.reset_coords(['num_contexts', 'range_length'])
            
            mapper = {c: f"{c}_{offset_val}" for c in y.data_vars}
            y = y.rename_vars(mapper)
            
            sixbase_over_tss.append(y)
            
            #print(offset_val, mod)
            
    # create data frame with summarised 6-base data over TSS
    sixbase_over_tss_results = xr.merge(sixbase_over_tss, compat = 'override')
    sixbase_over_tss_df = sixbase_over_tss_results.to_dataframe().droplevel(1)
    sixbase_over_tss_df = sixbase_over_tss_df.drop(["contig", "start", "end"], axis=1)

    # add atac data to df
    result_df = atac_over_tss.df.copy()
    result_df = result_df.set_index("Ranges_ID")
    result_df = result_df.join(sixbase_over_tss_df)
    # not including TSS with accessibility = 0 at this stage
    # TODO: identify "true" regions with 0 accessibility using coverage from WGS data?
    result_df['log_accessibility'] = [np.log(x) if x > 0 else np.nan for x in result_df.accessibility.values]
    return(result_df)
    

# for each region, get the mean accessibility, weighted by the length of the region
def collapse(df, val):
    x = np.average(df[val], weights = df.Length)
    return x


def offset(df, **kwargs):
    df_copy = df.copy()
    df_copy.loc[:, "End"] = kwargs["offset"] + df_copy.End
    df_copy.loc[:, "Start"] = kwargs["offset"] + df_copy.Start
    return df_copy

## map

In [10]:
beds250_m0={}
#beds250_m4={}
beds250_m8={}

for i in sets:
    print('-------------     Mapping: '+i+'     -------------')
    beds250_m0[i]=get_atac_6base_over_tss(time = 0,
                                          ds = ds,
                                          tss_pr = beds250[i])
#     beds250_m4[i]=get_atac_6base_over_tss(time = 4,
#                                           ds = ds,
#                                           tss_pr = beds250[i])
    beds250_m8[i]=get_atac_6base_over_tss(time = 8,
                                          ds = ds,
                                          tss_pr = beds250[i])

-------------     Mapping: all     -------------
Getting 6-base data for sample: 10274-AJ-0289
Loading ATAC-seq data from: /data/hodges_lab/Tim/biomodal/data/bedgraphs/NPCdiffATACme0hr_merge_filtered.bedGraph
Overlapping ATAC-seq data with TSS regions.
Summarising 6-base data over TSS regions.
mc
hmc
modc
Getting 6-base data for sample: 10274-AJ-0291
Loading ATAC-seq data from: /data/hodges_lab/Tim/biomodal/data/bedgraphs/NPCdiffATACme12day_merge_filtered.bedGraph
Overlapping ATAC-seq data with TSS regions.
Summarising 6-base data over TSS regions.
mc
hmc
modc


In [11]:
beds250_m0

{'all':           Chromosome     Start       End  accessibility   Start_b     End_b  \
 Ranges_ID                                                                     
 0               chr1    267764    268264       3.017969    267764    268264   
 1               chr1    631745    632245    4282.071431    631745    632245   
 2               chr1    778539    779039      35.589427    778539    779039   
 3               chr1    827223    827723       7.054094    827223    827723   
 4               chr1    844606    845106       4.541820    844606    845106   
 ...              ...       ...       ...            ...       ...       ...   
 99823          chr22  50775093  50775593      16.806213  50775093  50775593   
 99824          chr22  50783405  50783905      12.815731  50783405  50783905   
 99825          chr22  50796271  50796771       0.762135  50796271  50796771   
 99826          chr22  50806755  50807255      11.821018  50806755  50807255   
 99827          chr22  50807757  

# Train/Predict

## Helper function def

In [12]:
## Written by Biomodal

def eval_results(test_obs, pred_obs):
    
    mse = mean_squared_error(test_obs, pred_obs)
    r2 = r2_score(test_obs, pred_obs)
    spear = spearmanr(test_obs, pred_obs).correlation
    pears = pearsonr(test_obs, pred_obs).statistic

    return {"mse": mse, "r2": r2, "pearsonR": pears, "spearmanR": spear}




def find_optimal_parameters(res_train, mod="mc + hmc", test_contig="chr8"):

    X = res_train.loc[res_train.accessibility > 0]
    X_train = X[X.Chromosome != test_contig]
    y_train = X_train['log_accessibility']

    # select predictors to use in model
    if (mod == "modc") | (mod == "mc"):
        predictors = ["frac_" + mod + "_mean_" + str(x) for x in [-1000,-500,0,500,1000]]
        predictors += ["num_contexts_" + str(x) for x in [-1000,-500,0,500,1000]]
    elif mod == "mc + hmc":
        predictors = ["frac_mc" + "_mean_" + str(x) for x in [-1000,-500,0,500,1000]]
        predictors += ["frac_hmc" + "_mean_" + str(x) for x in [-1000,-500,0,500,1000]]
        predictors += ["num_contexts_" + str(x) for x in [-1000,-500,0,500,1000]]
    
#     param_grid = {
#         "n_estimators": [500],
#         "max_depth": [7],
#         "eta": [0.01],
#         "subsample": np.arange(0.5, 0.6, 0.1),
#         "colsample_bytree": [0.8],
#     }
    
    param_grid = {
        "n_estimators": np.arange(100, 600, 200),
        "max_depth": np.arange(3, 8, 2),
        "eta": np.arange(0.01, 0.05, 0.01),
        "subsample": np.arange(0.2, 0.6, 0.1),
        "colsample_bytree": np.arange(0.8, 1.0, 0.05),
    }
    
    #{'colsample_bytree': 0.8, 'eta': 0.01, 'max_depth': 7, 'n_estimators': 500, 'subsample': 0.5000000000000001}
    
    print("Finding optimal parameters for: " + mod)
    regressor = xgb.XGBRegressor(eval_metric="rmsle")
    search = GridSearchCV(regressor, param_grid, cv=3).fit(X_train[predictors], y_train)
    print("Optimal parameters for " + mod + ":")
    print(search.best_params_)

## Find optimal parameters

###  Timepoint: 0 hrs 

In [13]:
find_optimal_parameters(beds250_m0["all"], test_contig="chr1")

Finding optimal parameters for: mc + hmc
Optimal parameters for mc + hmc:
{'colsample_bytree': 0.9500000000000002, 'eta': 0.02, 'max_depth': 7, 'n_estimators': 500, 'subsample': 0.5000000000000001}


###  Timepoint: 8 days 

In [13]:
find_optimal_parameters(beds250_m8["all"], test_contig="chr1")

Finding optimal parameters for: mc + hmc
Optimal parameters for mc + hmc:
{'colsample_bytree': 0.9500000000000002, 'eta': 0.02, 'max_depth': 7, 'n_estimators': 500, 'subsample': 0.5000000000000001}
